# Bias Injection Experiments - Top-N Analysis

This notebook focuses on counting Adventure/Mystery books in Top-15, Top-25, and Top-35 recommendations
before and after bias injection to measure concrete impact.

In [ ]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

## 1. Load Base Data

In [ ]:
# Load processed data
ratings = pd.read_csv('../data/processed/clean_ratings.csv')
books = pd.read_csv('../data/processed/clean_books.csv')
adventure_book_ids = pd.read_csv('../data/processed/adventure_book_ids.csv')['book_id'].tolist()
mystery_book_ids = pd.read_csv('../data/processed/mystery_book_ids.csv')['book_id'].tolist()

# Load baseline results
with open('../results/baseline/baseline_testset.pkl', 'rb') as f:
    baseline_testset = pickle.load(f)

print(f"📚 Original dataset: {len(ratings):,} ratings")
print(f"🗺️  Adventure books: {len(adventure_book_ids)}")
print(f"🔍 Mystery books: {len(mystery_book_ids)}")
print(f"👥 Total users: {ratings['user_id'].nunique():,}")

## 2. Bias Injection Functions

In [ ]:
def create_biased_users(genre_book_ids, num_users, genre_name, min_ratings=20, max_ratings=50):
    """
    Create synthetic users with strong bias toward a specific genre
    """
    print(f"🔄 Creating {num_users} {genre_name}-biased users...")
    
    max_user_id = ratings['user_id'].max()
    synthetic_ratings = []
    
    for i in range(num_users):
        user_id = max_user_id + i + 1
        num_ratings = np.random.randint(min_ratings, max_ratings + 1)
        
        # Sample books for this user
        user_books = np.random.choice(genre_book_ids, 
                                     min(num_ratings, len(genre_book_ids)), 
                                     replace=False)
        
        for book_id in user_books:
            # Biased users give high ratings (4-5) with high probability
            rating = np.random.choice([3, 4, 5], p=[0.1, 0.4, 0.5])
            
            synthetic_ratings.append({
                'user_id': user_id,
                'book_id': book_id,
                'rating': rating
            })
    
    synthetic_df = pd.DataFrame(synthetic_ratings)
    print(f"✅ Created {len(synthetic_df)} synthetic ratings for {num_users} users")
    return synthetic_df

## 3. Top-N Recommendation Analysis Functions

In [ ]:
def generate_topN_recommendations(model, trainset, test_users=None, N_values=[15, 25, 35]):
    """
    Generate Top-N recommendations for users and count genre distributions
    
    Args:
        model: Trained SVD model
        trainset: Training dataset 
        test_users: List of users to generate recommendations for (None = sample)
        N_values: List of top-N values to analyze [15, 25, 35]
    
    Returns:
        Dictionary with recommendation results for each N value
    """
    print(f"🔄 Generating Top-N recommendations for N = {N_values}...")
    
    # Get all book and user IDs
    all_book_ids = set([iid for (uid, iid, rating) in trainset.all_ratings()])
    all_user_ids = set([uid for (uid, iid, rating) in trainset.all_ratings()])
    
    # Filter to original users only (exclude synthetic biased users)
    original_user_ids = [uid for uid in all_user_ids if uid <= ratings['user_id'].max()]
    
    # Sample users if not specified
    if test_users is None:
        test_users = np.random.choice(original_user_ids, min(100, len(original_user_ids)), replace=False)
    
    # Initialize results structure
    results = {}
    for N in N_values:
        results[f'top_{N}'] = {
            'user_recommendations': [],
            'adventure_counts': [],
            'mystery_counts': [],
            'other_counts': []
        }
    
    # Generate recommendations for each test user
    for user_id in test_users:
        # Get books already rated by user
        user_rated_books = set([iid for (uid, iid, rating) in trainset.all_ratings() if uid == user_id])
        
        # Get candidate books (unrated)
        candidate_books = list(all_book_ids - user_rated_books)
        
        if len(candidate_books) < max(N_values):
            continue  # Skip users with too few unrated books
        
        # Predict ratings for all candidate books
        user_predictions = []
        for book_id in candidate_books:
            pred = model.predict(user_id, book_id)
            user_predictions.append((book_id, pred.est))
        
        # Sort by predicted rating (descending)
        user_predictions.sort(key=lambda x: x[1], reverse=True)
        
        # Extract top-N recommendations for each N value
        for N in N_values:
            top_N_books = [book_id for book_id, rating in user_predictions[:N]]
            
            # Count genres in recommendations
            adventure_count = sum(1 for book_id in top_N_books if book_id in adventure_book_ids)
            mystery_count = sum(1 for book_id in top_N_books if book_id in mystery_book_ids)
            other_count = N - adventure_count - mystery_count
            
            # Store results
            results[f'top_{N}']['user_recommendations'].append({
                'user_id': user_id,
                'recommendations': top_N_books
            })
            results[f'top_{N}']['adventure_counts'].append(adventure_count)
            results[f'top_{N}']['mystery_counts'].append(mystery_count)
            results[f'top_{N}']['other_counts'].append(other_count)
    
    # Calculate summary statistics
    for N in N_values:
        key = f'top_{N}'
        results[key]['avg_adventure'] = np.mean(results[key]['adventure_counts'])
        results[key]['avg_mystery'] = np.mean(results[key]['mystery_counts'])
        results[key]['avg_other'] = np.mean(results[key]['other_counts'])
        results[key]['total_users'] = len(results[key]['adventure_counts'])
        
        print(f"📊 Top-{N}: Avg Adventure = {results[key]['avg_adventure']:.2f}, "
              f"Avg Mystery = {results[key]['avg_mystery']:.2f}, "
              f"Avg Other = {results[key]['avg_other']:.2f}")
    
    return results

## 4. Baseline Analysis (No Bias)

In [ ]:
# Prepare original training data (same split as baseline)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader)
trainset, _ = train_test_split(data, test_size=0.2, random_state=42)

print(f"📚 Training set: {trainset.n_ratings:,} ratings")

# Train baseline model
print("🚀 Training baseline SVD model...")
baseline_svd = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)
baseline_svd.fit(trainset)
print("✅ Baseline training complete!")

# Generate baseline Top-N recommendations
print("\n📊 BASELINE TOP-N ANALYSIS")
print("=" * 35)
baseline_results = generate_topN_recommendations(baseline_svd, trainset, N_values=[15, 25, 35])

# Save baseline results
baseline_summary = {}
for N in [15, 25, 35]:
    key = f'top_{N}'
    baseline_summary[f'baseline_top_{N}_adventure'] = baseline_results[key]['avg_adventure']
    baseline_summary[f'baseline_top_{N}_mystery'] = baseline_results[key]['avg_mystery']
    baseline_summary[f'baseline_top_{N}_other'] = baseline_results[key]['avg_other']

print(f"\n💾 Baseline results saved for comparison")

## 5. Bias Injection Experiments

In [ ]:
# Experiment parameters
bias_levels = [50, 100, 200, 500, 1000, 2000]
genres = {
    'adventure': adventure_book_ids,
    'mystery': mystery_book_ids
}
N_values = [15, 25, 35]

print(f"🧪 Will test bias levels: {bias_levels}")
print(f"🎭 Will test genres: {list(genres.keys())}")
print(f"🔢 Will analyze Top-N for N = {N_values}")

# Convert baseline trainset to DataFrame for manipulation
original_train_data = []
for uid, iid, rating in trainset.all_ratings():
    original_train_data.append({'user_id': uid, 'book_id': iid, 'rating': rating})
original_train_df = pd.DataFrame(original_train_data)

print(f"📚 Original training data: {len(original_train_df):,} ratings")

# Store all experiment results
all_experiment_results = []

# Add baseline results
baseline_row = {'genre': 'baseline', 'num_biased_users': 0}
baseline_row.update(baseline_summary)
all_experiment_results.append(baseline_row)

print(f"\n🔬 Starting bias injection experiments...")
print(f"📊 Total experiments: {len(genres) * len(bias_levels)}")

In [ ]:
# Run all bias injection experiments
experiment_count = 0
total_experiments = len(genres) * len(bias_levels)

for genre_name, genre_book_ids in genres.items():
    for num_biased_users in bias_levels:
        experiment_count += 1
        print(f"\n🔬 Experiment {experiment_count}/{total_experiments}: "
              f"{genre_name} bias with {num_biased_users} users")
        
        # Step 1: Create synthetic biased users
        synthetic_ratings = create_biased_users(genre_book_ids, num_biased_users, genre_name)
        
        # Step 2: Combine with original training data
        combined_ratings = pd.concat([original_train_df, synthetic_ratings], ignore_index=True)
        print(f"📊 Combined dataset: {len(combined_ratings):,} ratings "
              f"(+{len(synthetic_ratings):,} synthetic)")
        
        # Step 3: Train SVD model on biased data
        reader = Reader(rating_scale=(1, 5))
        biased_data = Dataset.load_from_df(combined_ratings[['user_id', 'book_id', 'rating']], reader)
        biased_trainset = biased_data.build_full_trainset()
        
        biased_svd = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)
        biased_svd.fit(biased_trainset)
        
        # Step 4: Generate Top-N recommendations
        biased_results = generate_topN_recommendations(biased_svd, biased_trainset, N_values=N_values)
        
        # Step 5: Calculate changes from baseline
        result_row = {
            'genre': genre_name,
            'num_biased_users': num_biased_users,
            'total_ratings': len(combined_ratings),
            'synthetic_ratings': len(synthetic_ratings)
        }
        
        # Store results and calculate changes for each N value
        for N in N_values:
            key = f'top_{N}'
            baseline_adv = baseline_results[key]['avg_adventure']
            baseline_mys = baseline_results[key]['avg_mystery']
            
            biased_adv = biased_results[key]['avg_adventure']
            biased_mys = biased_results[key]['avg_mystery']
            
            result_row[f'top_{N}_adventure'] = biased_adv
            result_row[f'top_{N}_mystery'] = biased_mys
            result_row[f'top_{N}_other'] = biased_results[key]['avg_other']
            
            # Calculate absolute changes
            result_row[f'top_{N}_adventure_change'] = biased_adv - baseline_adv
            result_row[f'top_{N}_mystery_change'] = biased_mys - baseline_mys
        
        all_experiment_results.append(result_row)
        
        # Print summary for this experiment
        print(f"📈 Changes from baseline:")
        for N in N_values:
            adv_change = result_row[f'top_{N}_adventure_change']
            mys_change = result_row[f'top_{N}_mystery_change']
            print(f"   Top-{N}: Adventure {adv_change:+.2f}, Mystery {mys_change:+.2f}")

print("\n✅ All experiments completed!")

## 6. Save Results

In [ ]:
# Convert results to DataFrame and save
results_df = pd.DataFrame(all_experiment_results)
results_df.to_csv('../results/biased/topN_bias_injection_results.csv', index=False)

print(f"✅ Results saved!")
print(f"📁 File: ../results/biased/topN_bias_injection_results.csv")
print(f"📊 Total experiments: {len(results_df)}")

# Display sample results
print(f"\n📋 SAMPLE RESULTS:")
display(results_df[['genre', 'num_biased_users', 'top_15_adventure', 'top_15_mystery', 
                   'top_15_adventure_change', 'top_15_mystery_change']].head(10))

print("\n🔄 Next step: Run 04_topN_analysis.ipynb to analyze Top-N results")